In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


In [7]:

class GraphAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(GraphAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[-1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(GraphAttentionLayer, self).build(input_shape)

    def call(self, x, adj_matrix):
        h = tf.matmul(x, self.kernel)
        # print(h.shape)
        attn_coef = tf.matmul(tf.matmul(h, adj_matrix), h, transpose_b=True)
        attn_coef = tf.nn.leaky_relu(attn_coef, alpha=0.2)
        attn_coef = tf.nn.softmax(attn_coef, axis=-1)
        output = tf.matmul(attn_coef, h)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

# Create a Keras Graph model with Graph Attention Layer
class GraphAutoencoderWithAttention(tf.keras.Model):
    def __init__(self, input_size, hidden_size):
        super(GraphAutoencoderWithAttention, self).__init__()

        # Graph Attention Layer
        self.attention = GraphAttentionLayer(output_dim=hidden_size)

        # Decoder
        self.decoder = layers.Dense(input_size, activation='sigmoid')

    def call(self, inputs, adj_matrix, training=None, mask=None):
        x = self.attention(inputs, adj_matrix)
        x = self.decoder(x)
        return x


In [8]:

# Define a simple graph (you can replace this with your own graph data)
adjacency_matrix = np.array([[0, 1, 0, 1, 2], [1, 0, 1, 2, 3],  [0, 1, 0, 3, 4], [1, 0, 1, 2, 5], [1, 0, 1, 2, 5]], dtype=np.float32)
node_features = np.array([[1.0,2.0], [2.0,1.0], [3.0,3.0], [4.0,2.0], [5.0,1.0], [1,2], [3,1]], dtype=np.float32)


In [9]:

# Instantiate the model
input_size = 2  # Size of node features
hidden_size = 5
model = GraphAutoencoderWithAttention(input_size, hidden_size)

# Define the loss function and optimizer
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


In [10]:

# Convert numpy arrays to TensorFlow tensors
adjacency_matrix_tensor = tf.constant(adjacency_matrix)
node_features_tensor = tf.constant(node_features)

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        reconstructed_features = model(node_features_tensor,adjacency_matrix_tensor)
        loss = loss_fn(node_features_tensor, reconstructed_features)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    if epoch % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.numpy():.4f}')

# After training, you can use the learned embeddings for downstream tasks
learned_embeddings = model(node_features_tensor, adjacency_matrix_tensor).numpy()
print(learned_embeddings)

Epoch 1/100, Loss: 4.2461
Epoch 11/100, Loss: 3.1633
Epoch 21/100, Loss: 2.9898
Epoch 31/100, Loss: 2.9544
Epoch 41/100, Loss: 2.9439
Epoch 51/100, Loss: 2.9397
Epoch 61/100, Loss: 2.9374
Epoch 71/100, Loss: 2.9360
Epoch 81/100, Loss: 2.9350
Epoch 91/100, Loss: 2.9342
[[0.9981042  0.9971571 ]
 [0.9982247  0.99729365]
 [0.998242   0.9973133 ]
 [0.9982425  0.9973138 ]
 [0.9982425  0.9973139 ]
 [0.9981042  0.9971571 ]
 [0.99824107 0.9973123 ]]
